In [1]:
import pexpect
import time
import sys
import re
import datetime
import os

In [2]:
#ENM3 -> 10.23.202.28 PW : "Enm#2021"
#ENM5 -> 10.167.30.28 PW : "Enm#2021"

child = pexpect.spawn('ssh -L 7899:localhost:8008 -J root@150.236.203.198 root@127.0.0.1 -p 8008', timeout=30, encoding='utf-8', logfile=sys.stdout)

child.maxread = 300000 

child.expect("root@150.236.203.198's password: ")
child.sendline('PMT@2022')
child.expect("root@127.0.0.1's password: ")
child.sendline('PMT@2022')

time.sleep(10)

# Shell command to take after the dates in modification dates
# child.sendline('find /var/opt/common5/mdt/ -type f -newermt 2023-04-16 -exec ls -l {} \;')

# Shell command to take only the folder
# child.sendline('find /var/opt/common5/mdt/20230205 -type f -exec ls -l {} \;')

# Shell command to take certain range dates in modification dates
child.sendline('find /var/opt/common5/mdt -type f -newermt 2023-03-01 ! -newermt 2023-06-03 -exec ls -l {} \;')

try:
    child.expect(pexpect.EOF, timeout=50)

except pexpect.TIMEOUT:
    child.close()
    raw = str(child.before)

root@150.236.203.198's password: PMT@2022

root@127.0.0.1's password: PMT@2022
find /var/opt/common5/mdt -type f -newermt 2023-03-01 ! -newermt 2023-06-03 -exec ls -l {} \;

Last login: Fri Jun  2 21:51:27 2023 from 192.168.21.6
! -newermt 2023-06-03 -exec ls -l {} \;-2 ~]# find /var/opt/common5/mdt -type f -newermt 2023-03-01  
-rw-r--r--. 1 root root 1690348 Mar 14 04:33 /var/opt/common5/mdt/20230314/00/result_ue_cap.zip
-rw-r--r--. 1 root root 28803113 Mar 14 04:35 /var/opt/common5/mdt/20230314/00/ue_traffic.zip
-rw-r--r--. 1 root root 58262073 Mar 14 04:37 /var/opt/common5/mdt/20230314/00/ue_meas.zip
-rw-r--r--. 1 root root 37523832 Mar 14 04:41 /var/opt/common5/mdt/20230314/00/mdt_result.zip
-rw-r--r--. 1 root root 705540325 Mar 14 05:17 /var/opt/common5/mdt/20230314/00/ta_histogram.zip
-rw-r--r--. 1 root root 1681850 Mar 14 05:20 /var/opt/common5/mdt/20230314/01/result_ue_cap.zip
-rw-r--r--. 1 root root 23916727 Mar 14 05:21 /var/opt/common5/mdt/20230314/01/ue_traffic.zip
-rw-r--

In [ ]:
test1 = re.split(r'\s+|\n', raw)

In [ ]:
# Create list comprehension to filter list of zip files only

# result = [file for file in test1 if file.endswith('.zip')]

# Create list comprehension to filter only mdt_result.zip only

pattern = r'.*/(?!filter_)[^/]+/mdt_result\.zip$'
result = [path for path in test1 if re.match(pattern, path)]

In [ ]:
# Create list comprehension to generate weekend dates in May

weekend_dates = [date.strftime("%Y%m%d") for date in (datetime.date(2023, 3, day) for day in range(1, 32)) 
                 if date.weekday() >= 5]

In [ ]:
# Create list comprehension to filter May Month only
result = [path for path in result if '202303' in path]

# Create list comprehension to filter Out Some Date
# result = [path for path in result if '20230501' not in path and '20230518' not in path]

# Create list comprehension to filter Out Some Date
result = [path for path in result if '20230322' not in path]

# Create list comprehension to filter Out Weekend Date
result = [path for path in result if path.split('/')[5] not in weekend_dates]

# Create list comprehension to filter Busy Hour only
result = [path for path in result if '/07/' in path or '/08/' in path or '/09/' in path or '/17/' in path or '/18/' in path or '/19/' in path]

# result = [path for path in result if '/07/' in path or '/08/' in path or '/09/']

In [ ]:
with open(r'/home/nivag/location_to_dl_8008.txt', 'w') as fp:
    fp.write('\n'.join(result))